In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [6]:
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [9]:
legit = data[data.Class == 0 ]

# Under sampling 
legit = legit.sample(n=492)
legit.shape

frau = data[data.Class == 1]
print(legit.shape, frau.shape)

(492, 31) (492, 31)


In [12]:
# Concate legit and fraud

data = pd.concat([legit, frau], axis = 0)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
62728,50405.0,-0.766150,1.081704,1.514567,0.866532,0.735766,0.669359,0.730121,-0.998741,-0.748557,...,0.810710,0.205641,-0.390050,-0.721545,0.529804,-0.040978,0.026551,-0.168728,58.50,0
224526,143844.0,2.089916,-1.992928,0.394606,-1.087858,-2.342715,0.079145,-2.002545,0.138239,0.040878,...,-0.197127,0.147582,0.268624,-0.010452,-0.466391,-0.179794,0.085686,-0.022918,61.24,0
54175,46320.0,1.088457,-0.900738,0.621114,-1.375277,-1.273040,-0.393514,-0.712383,0.128047,1.801553,...,0.186532,0.546569,-0.223365,0.075762,0.444792,0.110157,0.020699,0.023116,88.00,0
9365,13673.0,-0.511347,0.399935,2.021333,0.923996,0.478959,0.150226,0.378578,-0.116848,0.799566,...,-0.173703,0.013523,0.039991,0.220999,-0.491212,0.269713,-0.068391,-0.090953,33.82,0
167834,118909.0,-5.760458,-2.738149,-1.259277,-3.699111,-1.053574,-1.507868,-0.610677,0.849512,1.131618,...,-0.555568,0.476878,0.800697,-0.306309,0.881504,-0.882804,-0.498783,1.325482,63.04,0


In [14]:
# Train test Split

X = data.drop('Class', axis = 1)
Y = data['Class']
print(X.shape, Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 1)
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (984,)
(984, 30) (787, 30) (197, 30)


In [15]:
# Modeling 
 
from sklearn.linear_model import LogisticRegression


In [16]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [17]:
Y_pred = model.predict(X_test)
accuracy = metrics.accuracy_score(Y_test, Y_pred)
print(accuracy)

0.8883248730964467
